In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import scipy.stats as sps
from timeit import default_timer as timer

import sys
sys.path.append('..')
# import modules
from modules import path_solver, pointwise_path, full_path, utils
from modules import PD_gap, dual_sorted_L1_norm as dual_norm

from slope.solvers import admm, hybrid_cd, newt_alm, oracle_cd, prox_grad

# Toy examples

In [2]:
X=np.array([[2,1,0],[1,2,1]], dtype=float)
Lambda = np.array([6,4,2], dtype=float)
y = np.array([15,5], dtype=float)

In [3]:
# for Numba compilation
_ = full_path(X, y , Lambda, ratio=0., k_max=0., rtol_pattern=1e-6, atol_pattern = 1e-6, rtol_gamma=1e-6, split_max=1e1, log=0)

/Users/xdupuis/Research/slope-path/github/experiments/../modules/solvers.py:42: NumbaExperimentalFeatureWarning: Use of isinstance() detected. This is an experimental feature.
  m = face_pattern(grad, Lambda, rtol_pattern, atol_pattern); M = [m]


In [10]:
# t_start = timer()
Gamma, Sol, Primal, Gap, M, Split, T = full_path(X, y , Lambda, ratio=1., k_max=1e3, rtol_pattern=1e-6, atol_pattern = 1e-8, rtol_gamma=1e-12, split_max=1e1, log=1)
# time_path = timer() - t_start
# print(f'slope-path: {time_path:.2e}s elapsed')

node 0: gamma = 6.000, 6.45e-05s elapsed, gap = 0.00e+00
node 1: gamma = 5.000, 2.06e-04s elapsed, gap = -1.42e-14
node 2: gamma = 3.750, 2.50e-04s elapsed, gap = -1.42e-14
node 3: gamma = 0.417, 2.74e-04s elapsed, gap = -1.42e-14
node 4: gamma = 0.000, 2.93e-04s elapsed, gap = 0.00e+00


In [19]:
6/3.75

1.6

In [11]:
# for Numba compilation
_ = path_solver(X, y , Lambda, k_max=0., rtol_pattern=1e-6, atol_pattern=1e-6, rtol_gamma=1e-6, split_max=1e1, log=0)
_ = hybrid_cd(X, y, Lambda, fit_intercept=False, verbose=False, tol=1e-1)

In [20]:
gamma = dual_norm(X.T@y, Lambda)/3.75
t_start = timer()
sol, (primal, gap) = path_solver(X, y, gamma*Lambda, k_max=1e3, rtol_pattern=1e-6, atol_pattern = 1e-8, rtol_gamma=1e-12, split_max=1e1, log=0)
time = timer() - t_start
print(f'slope-path: {time:.2e}s elapsed') #, gap = {gap:.2e}') #, loss = {primal/X.shape[1]:.2e}')
print(sol)
print(primal/X.shape[0], gap/X.shape[0])

slope-path: 2.41e-04s elapsed
[5.08 0.   0.  ]
30.242000000000004 0.0


In [22]:
t_start = timer()
sol_cd, intercept_cd, primals_cd, gaps_cd, time_cd, n_cluster = hybrid_cd(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, verbose=False, tol=1e-13)
time_cd = timer() - t_start
print(f'slope-cd: {time_cd:.2e}s elapsed')
print(sol_cd)
print(primals_cd[-1], gaps_cd[-1])
primal_cd, gap_cd = PD_gap(sol_cd, X, y, gamma*Lambda)
# , gap = {gap_cd:.2e}')
print(primal_cd/X.shape[0], gap_cd/X.shape[0])
# #print(primal, sol)
# print(primals_cd[-1], f'{gaps_cd[-1]:.2e}')

slope-cd: 3.74e-04s elapsed
[ 5.08  0.   -0.  ]
30.242000000000004 0.0
30.242000000000004 0.0


In [8]:
res = y - X@sol_cd
print(dual_norm(X.T@res, gamma*Lambda) <= 1)

primal_cd - 1/2*(np.linalg.norm(y)**2 - np.linalg.norm(y - res)**2)
print(PD_gap(sol_cd, X, y, gamma*Lambda))


True
(96.1, -2.842170943040401e-14)


In [9]:
gamma = 4.# use float
t_start = timer()
sol, (primal, gap) = pointwise_path(X, y , Lambda, gamma, k_max=1e3, rtol_pattern=1e-6, atol_pattern = 1e-8, rtol_gamma=1e-12, split_max=1e1, log=0)
time = timer() - t_start
print(f'slope-path: {time:.2e}s elapsed') #, gap = {gap:.2e}') #, loss = {primal/X.shape[1]:.2e}')
print(sol)
print(primal/X.shape[0], gap/X.shape[0])

TypingError: Failed in nopython mode pipeline (step: Handle with contexts)
Failed in nopython mode pipeline (step: nopython frontend)
Internal error at <numba.core.typeinfer.CallConstraint object at 0x169ff1760>.
too many positional arguments
During: resolving callee type: type(CPUDispatcher(<function PD_gap at 0x11eeead30>))
During: typing of call at /Users/xdupuis/Research/slope-path/github/experiments/../modules/solvers.py (154)

Enable logging at debug level for details.

File "../modules/solvers.py", line 154:
def pointwise_path(X, y, Lambda, gamma, k_max, rtol_pattern, atol_pattern, rtol_gamma, split_max, log):
    <source elided>
            # print('node', k,': running gamma =', run_gamma)
    return sol, PD_gap(sol, gamma, X, y, Lambda)
    ^


In [ ]:
eps=np.sqrt(2e-10)
X = np.array([[2,1,0],[1,2,1],[eps,0,0],[0,eps,0],[0,0,eps]], dtype=float)
Lambda = np.array([6,4,2], dtype=float)
y = np.array([15, 5,0,0,0], dtype=float)

In [ ]:
gamma = 2. # use float
t_start = timer()
sol, (primal, gap) = pointwise_path(X, y , Lambda, gamma, k_max=1e3, rtol_pattern=1e-6, atol_pattern = 1e-8, rtol_gamma=1e-12, split_max=1e1, log=1)
time = timer() - t_start
print(f'slope-path: {time:.2e}s elapsed, gap = {gap:.2e}')

node 0: running gamma = 6.000
node 1: running gamma = 5.000
node 2: running gamma = 3.750
node 3: running gamma = 2.000
slope-path: 3.31e-03s elapsed, gap = 1.42e-14


In [13]:
# t_start = timer()
Gamma, Sol, Primal, Gap, M, Split, T = full_path(X, y , Lambda, ratio=1., k_max=1e3, rtol_pattern=1e-6, atol_pattern = 1e-8, rtol_gamma=1e-12, split_max=1e1, log=1)
# time_path = timer() - t_start
# print(f'slope-path: {time_path:.2e}s elapsed')

node 0: gamma = 6.000, 2.17e-05s elapsed, gap = 0.00e+00
node 1: gamma = 5.000, 1.00e-04s elapsed, gap = 0.00e+00
node 2: gamma = 3.750, 1.36e-04s elapsed, gap = -2.84e-14
node 3: gamma = 0.417, 1.63e-04s elapsed, gap = -4.26e-14
node 4: gamma = 0.000, 1.86e-04s elapsed, gap = 6.60e-09


# Ribofalvin

In [50]:
ribo = pd.read_csv("riboflavin.csv", sep=',', index_col=0).T
ribo

,q_RIBFLV,AADK_at,AAPA_at,ABFA_at,ABH_at,ABNA_at,ABRB_at,ACCA_at,ACCB_at,ACCC_at,...,YYDF_i_at,YYDG_at,YYDH_at,YYDI_at,YYDJ_at,YYDK_at,YYXA_at,YYZB_at,YYZE_at,zur_at
b_Fbat107PT24.CEL,-6.643856,8.492404,8.111451,8.320842,10.287112,8.261279,10.208273,9.745474,9.818821,9.676227,...,7.882737,5.440163,4.981684,6.481998,5.363919,7.773494,7.668711,3.762119,7.172823,7.427105
b_Fbat107PT30.CEL,-6.947862,7.639380,7.239965,7.289051,9.862287,7.303497,9.500023,9.216008,9.854946,9.650078,...,7.912135,5.383739,4.597438,6.390127,4.763876,7.336261,7.600104,3.761616,7.187113,7.545287
b_Fbat107PT48.CEL,-7.930160,8.088340,7.855510,7.793395,9.676720,7.098273,9.473917,9.580384,9.926077,9.787129,...,7.354618,5.718795,4.468228,6.123498,5.400686,7.573388,7.873352,3.741953,7.251285,7.718321
b_Fbat107PT52.CEL,-8.287712,7.886820,7.939513,7.997588,9.680562,7.408494,9.788725,9.447722,9.852773,9.546915,...,7.691985,5.561048,4.787030,5.904708,5.139692,7.214269,7.384115,3.887179,7.117375,7.260654
knh_102_Fbat289PT24.CEL,-7.310432,6.805762,7.554522,7.609902,8.551953,7.712407,8.490847,8.696249,8.573272,8.589661,...,7.999088,5.549914,5.258139,6.206778,4.910169,6.884474,6.996956,4.034177,7.722534,6.828164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
knhb_252_Fbat526PT48.CEL,-8.333516,6.669958,7.731588,7.719349,8.267227,6.821443,8.482042,8.231037,8.428432,8.497505,...,8.646136,5.318534,4.908785,6.081095,5.284497,6.802258,7.079083,4.492137,7.351626,6.903990
knhb_255_Fbat527PT24.CEL,-6.947862,7.833311,8.136090,8.165093,9.562706,8.242625,9.749418,9.360730,8.974610,9.028068,...,8.191385,4.991214,4.603681,5.887783,4.955115,6.531699,7.947857,3.954690,7.269928,7.439051
knhb_256_Fbat527PT48.CEL,-8.039785,6.163461,7.714285,7.560790,7.476788,6.972776,7.148842,7.373864,7.654946,7.818791,...,7.673101,5.492333,5.167516,6.089535,5.261065,6.925765,7.244050,4.421958,7.842281,6.892450
knhb_259_Fbat528PT24.CEL,-6.764150,7.342565,7.743141,7.709288,9.307739,7.999415,9.762122,9.105318,8.988289,9.004700,...,8.522762,5.407418,5.480406,6.552994,5.609084,7.148286,7.522179,4.986751,7.539126,7.309948


In [51]:
target = ribo['q_RIBFLV']
data = ribo.drop(columns='q_RIBFLV')

In [52]:
X = data.to_numpy(dtype=float)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0, ddof=0)
X = (X - X_mean) / X_std

y = target.to_numpy(dtype=float)
y_mean = y.mean()
y = y - y_mean 

Lambda = np.linspace(4,1,X.shape[-1],dtype=float)

avec 
- `rtol_pattern=1e-6, atol_pattern = 1e-8, rtol_gamma=1e-12` : 62 noeuds
- `rtol_pattern=1e-8, atol_pattern = 1e-8, rtol_gamma=1e-12` : 1574 noeuds (96%)
- `rtol_pattern=1e-10, atol_pattern = 1e-8, rtol_gamma=1e-12` : 5426 noeuds (99,8%)
- `rtol_pattern=1e-10, atol_pattern = 1e-8, rtol_gamma=1e-10` : 6303 noeuds (99,99%)

In [80]:
#Lambda=np.sqrt(range(1,X.shape[-1]+1))-np.sqrt(range(0,X.shape[-1]))
#Lambda = np.linspace(4,1,X.shape[-1],dtype=float)
q = 0.05
p = X.shape[-1]
t= np.linspace(1-q/(2*p),1-q/2,p)
Lambda = sps.norm.ppf(t) # Lambda BH 
# t_start = timer()
Gamma, Sol, Primal, Gap, M, Split, T = full_path(X, y , Lambda, ratio=0.9, k_max=1e4, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-12, split_max=1e2, log=1)
# time_path = timer() - t_start
# print(f'slope-path: {time_path:.2e}s elapsed')

node 0: gamma = 9.771, 5.25e-03s elapsed, gap = 0.00e+00
node 1: gamma = 9.155, 1.54e-02s elapsed, gap = 0.00e+00
node 2: gamma = 8.057, 2.10e-02s elapsed, gap = 7.11e-15
node 3: gamma = 7.898, 2.40e-02s elapsed, gap = 3.55e-15
node 4: gamma = 7.797, 2.87e-02s elapsed, gap = 3.55e-15
node 5: gamma = 7.793, 3.29e-02s elapsed, gap = 0.00e+00
node 6: gamma = 7.706, 3.57e-02s elapsed, gap = 0.00e+00
node 7: gamma = 7.605, 3.93e-02s elapsed, gap = 3.55e-15
node 8: gamma = 7.291, 4.31e-02s elapsed, gap = 7.11e-15
node 9: gamma = 7.137, 4.73e-02s elapsed, gap = 7.11e-15
node 10: gamma = 7.091, 5.04e-02s elapsed, gap = 0.00e+00
node 11: gamma = 7.039, 5.34e-02s elapsed, gap = 3.55e-15
node 12: gamma = 6.996, 5.77e-02s elapsed, gap = 0.00e+00
node 13: gamma = 6.905, 6.15e-02s elapsed, gap = -3.55e-15
node 14: gamma = 6.663, 6.49e-02s elapsed, gap = -3.55e-15
node 15: gamma = 6.598, 7.20e-02s elapsed, gap = 7.11e-15
node 16: gamma = 6.590, 9.63e-02s elapsed, gap = 0.00e+00
node 17: gamma = 6.440

In [54]:
1-Gamma[-1]/Gamma[0]

0.95

In [56]:
gamma = Gamma[-1]
gamma

0.22725787595946342

In [36]:
np.linalg.norm(Gap, ord=np.inf)

1.3655743202889425e-13

In [37]:
gamma = 5.5 # use float
t_start = timer()
sol, (primal, gap) = solver.pointwise(X, y , Lambda, gamma, k_max=1e3, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=1)
time = timer() - t_start
print(f'slope-path: {time:.2e}s elapsed, gap = {gap:.2e}')

node 0: running gamma = 11.363
node 1: running gamma = 10.383
node 2: running gamma = 9.979
node 3: running gamma = 8.788
node 4: running gamma = 7.498
node 5: running gamma = 6.761
node 6: running gamma = 6.751
node 7: running gamma = 6.718
node 8: running gamma = 6.526
node 9: running gamma = 6.514
node 10: running gamma = 6.506
node 11: running gamma = 6.495
node 12: running gamma = 6.322
node 13: running gamma = 6.306
node 14: running gamma = 5.833
node 15: running gamma = 5.823
node 16: running gamma = 5.526
node 17: running gamma = 5.502
node 18: running gamma = 5.500
slope-path: 7.61e-02s elapsed, gap = -3.55e-15


In [38]:
# t_start = timer()
Gamma1, *_ = solver.full_path(X, y , Lambda, ratio=0.95, k_max=1e4, rtol_pattern=1e-8, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=0)
# time_path = timer() - t_start
# print(f'slope-path: {time_path:.2e}s elapsed')

slope-path: 5.21e+00s elapsed


In [39]:
# t_start = timer()
Gamma2, *_ = solver.full_path(X, y , Lambda, ratio=0.95, k_max=1e4, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=0)
# time_path = timer() - t_start
# print(f'slope-path: {time_path:.2e}s elapsed')

slope-path: 5.94e+00s elapsed


In [40]:
np.array(Gamma1[:1000]) - np.array(Gamma2[:1000])


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [88]:
#gamma = 10.533/50.# use float
#gamma = 898.291/10
gamma = 9.771/2
t_start = timer()
sol, (primal, gap) = pointwise_path(X, y , Lambda, gamma, k_max=1e4, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=0)
time = timer() - t_start
print(f'slope-path: {time:.2e}s elapsed') #, gap = {gap:.2e}') #, loss = {primal/X.shape[1]:.2e}')
# print(sol)
print(primal/X.shape[0], gap/X.shape[0])

slope-path: 2.16e-01s elapsed
0.3354370653942376 -5.003822082817607e-17


In [86]:
#gamma = 10.533/50
#gamma = 898.291/10
gamma = 9.771/2
t_start = timer()
sol_cd, intercept_cd, primals_cd, gaps_cd, time_cd, n_cluster = hybrid_cd(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, verbose=False, tol=1e-15)
time_cd = timer() - t_start
print(f'slope-cd: {time_cd:.2e}s elapsed')
# print(sol_cd)
print(primals_cd[-1], gaps_cd[-1])
primal_cd, gap_cd = utils.PD_gap(sol_cd, gamma, X, y, Lambda)
# , gap = {gap_cd:.2e}')
print(primal_cd/X.shape[0], gap_cd/X.shape[0])
# #print(primal, sol)
# print(primals_cd[-1], f'{gaps_cd[-1]:.2e}')

slope-cd: 1.73e-01s elapsed
0.33543706539423773 2.7755575615628914e-16
0.3354370653942377 2.5019110414088036e-16


In [42]:
print(primal/X.shape[0])
print(primal_cd/X.shape[0])
print(primals_cd[-1])


0.3834194434407633
0.38341944344076323
0.38341944344076334


# Wine quality data set

In [15]:
wine = pd.read_csv("winequality-red.csv", sep=';')
wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [18]:
data = wine.drop(columns=['quality'])
target = wine['quality']

In [19]:
X = data.to_numpy(dtype=float)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0, ddof=0)
X = (X - X_mean) / X_std

y = target.to_numpy(dtype=float)
y_mean = y.mean()
y = y - y_mean

[[ 1599.          -409.55330074  1074.05379219   183.52798247
    149.83459293  -245.91691439  -180.97712743  1068.2076201
  -1092.08213312   292.62605663   -98.60756473]
 [ -409.55330074  1599.          -883.44059961     3.06669326
     98.01513819   -16.79561938   122.27553771    35.21994528
    375.66473376  -417.31770977  -323.45855542]
 [ 1074.05379219  -883.44059961  1599.           229.57988135
    325.91283921   -97.50402858    56.81730527   583.55053316
   -866.50472744   500.11930012   175.73529138]
 [  183.52798247     3.06669326   229.57988135  1599.
     88.91964679   299.09134317   324.64158283   568.0981102
   -136.95822313     8.83786702    67.2786241 ]
 [  149.83459293    98.01513819   325.91283921    88.91964679
   1599.             8.89387306    75.79334875   320.8110903
   -423.77678375   593.64550958  -353.60373112]
 [ -245.91691439   -16.79561938   -97.50402858   299.09134317
      8.89387306  1599.          1067.59865432   -35.09138403
    112.53362011    82.6004

In [20]:
Lambda = np.linspace(4,1,X.shape[-1],dtype=float)

In [37]:
Gamma, Sol, Primal, Gap, M, Split, T = full_path(X, y , Lambda, ratio=1., k_max=1e4, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=1)


node 0: gamma = 153.671, 1.10e-04s elapsed, gap = 0.00e+00
node 1: gamma = 131.390, 6.11e-04s elapsed, gap = 1.14e-13
node 2: gamma = 74.536, 9.09e-04s elapsed, gap = 1.14e-13
node 3: gamma = 47.619, 1.21e-03s elapsed, gap = 2.84e-13
node 4: gamma = 37.204, 1.49e-03s elapsed, gap = 5.68e-14
node 5: gamma = 35.464, 1.82e-03s elapsed, gap = 5.68e-14
node 6: gamma = 32.990, 2.16e-03s elapsed, gap = 3.41e-13
node 7: gamma = 32.653, 2.54e-03s elapsed, gap = 1.71e-13
node 8: gamma = 31.224, 2.87e-03s elapsed, gap = 1.71e-13
node 9: gamma = 28.891, 3.24e-03s elapsed, gap = 1.71e-13
node 10: gamma = 18.967, 3.57e-03s elapsed, gap = 1.71e-13
node 11: gamma = 15.255, 3.95e-03s elapsed, gap = 1.14e-13
node 12: gamma = 12.299, 4.27e-03s elapsed, gap = 1.71e-13
node 13: gamma = 11.986, 4.66e-03s elapsed, gap = 1.71e-13
node 14: gamma = 9.983, 5.10e-03s elapsed, gap = 1.14e-13
node 15: gamma = 7.755, 6.51e-03s elapsed, gap = 2.27e-13
node 16: gamma = 7.244, 1.30e-02s elapsed, gap = 1.14e-13
node 17:

In [27]:
gamma = 15.3671/5 #76.8355 # use float
t_start = timer()
sol, (primal, gap) = pointwise_path(X, y , Lambda, gamma, k_max=1e4, rtol_pattern=1e-10, atol_pattern = 1e-10, rtol_gamma=1e-10, split_max=1e1, log=0)
time = timer() - t_start
print(primal)
print(sol)
#print(gap)
print(f'slope-path: {time:.2e}s elapsed') #, gap = {gap:.2e}') #, loss = {primal/X.shape[1]:.2e}')
# print(sol)
#print(primal/X.shape[0], gap/X.shape[0])

#5.625784e-15 -1.028374e-01  3.576910e-15 -1.871090e-15 -1.361365e-15
# 1.286609e-15 -2.431625e-15  3.144004e-15 -3.323674e-15 -1.521597e-15  1.714055e-01


343.50061270247767
[ 0.03374243 -0.18651895 -0.01788864  0.01788864 -0.0822663   0.03374243
 -0.09521569 -0.0287343  -0.05428365  0.14639657  0.29006365]
slope-path: 4.16e-03s elapsed


In [31]:
gamma = 15.3671/5 #76.8355
t_start = timer()
sol_cd, intercept_cd, primals_cd, gaps_cd, time_cd, n_cluster = hybrid_cd(X, y, gamma/X.shape[0]*Lambda, fit_intercept=False, verbose=False, tol=1e-15)
time_cd = timer() - t_start
print(f'slope-cd: {time_cd:.2e}s elapsed')
# print(sol_cd)
print(primals_cd[-1], gaps_cd[-1])
primal_cd, gap_cd = utils.PD_gap(sol_cd, gamma, X, y, Lambda)
# , gap = {gap_cd:.2e}')
print(primal_cd/X.shape[0], gap_cd/X.shape[0])
# #print(primal, sol)
# print(primals_cd[-1], f'{gaps_cd[-1]:.2e}')

slope-cd: 1.85e-02s elapsed
0.2148221467807865 -8.326672684688674e-17
0.21482214678078654 2.1329613080978617e-16


In [69]:
X.shape

(1599, 11)